In [1]:
from pathlib import Path
import typing as T
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy
import xarray as xr
import iris
import iris.plot
import numpy as np
import os
import sys
import textwrap
import itertools
import ants
import warnings

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/ants/regrid/esmf.py:26: UserWarning:  No module named 'ESMF'
Proceeding without capabilities provided by ESMPy (ESMF).
  warnings.warn(msg.format(str(_ESMF_IMPORT_ERROR)))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/ants/regrid/_ugrid.py:19: UserWarning:  No module named 'ESMF'
Proceeding without capabilities provided by ESMPy (ESMF).
  warnings.warn(msg.format(str(_ESMF_IMPORT_ERROR)))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/experimental/raster.py:29: IrisDeprecation: iris.experimental.raster is deprecated since version 3.2, and will be removed in a future release. If you make use of this functionality, please contact the Iris Developers to discuss how to retain it (which may involve reversing the deprecation).
  warn_deprecated(wmsg)


In [2]:
PATH = Path('/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um')

In [3]:
files = PATH.glob('*.nc')

In [4]:
for file in files:
    print (file)

/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverd012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pvera012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverc012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pb000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pb012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverb012.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pverc000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_pvera000.nc
/home/548/pag548/cylc-run/u-dq126/share/cycle/20220227T0000Z/Flagship_ERA5to1km/5km/RAL3P2/um/umnsaa_p

In [41]:
files = PATH.glob('*.nc')
long_names = []
var_names = []
for file in files:
    cubes = iris.load(file)
    for cube in cubes:
        print (f'{file.name} {cube.long_name} {cube.standard_name} {cube.var_name}')
        long_names.append(cube.long_name)
        var_names.append(cube.var_name)

umnsaa_pverd012.nc TEMPERATURE ON P LEV/P GRID air_temperature STASH_m01s16i203_2
umnsaa_pverd012.nc GEOPOTENTIAL HEIGHT ON P LEV/P GRID geopotential_height STASH_m01s16i202_2
umnsaa_pverd012.nc RH WRT ICE ON P LEV/P GRID relative_humidity STASH_m01s16i204_2
umnsaa_pvera012.nc CUMULUS-CAPPED BL INDICATOR None STASH_m01s03i310
umnsaa_pvera012.nc PROBABILITY OF VIS LESS THAN 5 KM None STASH_m01s03i253
umnsaa_pvera012.nc TURBULENT MIXING HT AFTER B.LAYER m None STASH_m01s03i304_2
umnsaa_pvera012.nc TEMPERATURE AT 1.5M air_temperature STASH_m01s03i236_2
umnsaa_pvera012.nc BOUNDARY LAYER DEPTH AFTER TIMESTEP atmosphere_boundary_layer_thickness STASH_m01s00i025
umnsaa_pvera012.nc DEWPOINT AT 1.5M (K) dew_point_temperature STASH_m01s03i250_2
umnsaa_pvera012.nc FOG FRACTION AT 1.5 M fog_area_fraction STASH_m01s03i248_2
umnsaa_pvera012.nc RELATIVE HUMIDITY AT 1.5M relative_humidity STASH_m01s03i245_2
umnsaa_pvera012.nc FRAC OF SEA ICE IN SEA AFTER TSTEP sea_ice_area_fraction STASH_m01s00i031
um

In [45]:
unique_long_names = (list(set(long_names)))
print (unique_long_names)

['FRAC OF SEA ICE IN SEA AFTER TSTEP', '10 METRE WIND U-COMP         B GRID', 'PRESSURE AT MEAN SEA LEVEL', 'TOTAL COLUMN WET MASS  RHO GRID', 'TURBULENT MIXING HT AFTER B.LAYER m', 'TOTAL DOWNWARD SURFACE SW FLUX', 'DEEP SOIL TEMP. AFTER HYDROLOGY DEGK', 'TEMPERATURE ON P LEV/P GRID', 'NET DOWN SURFACE SW FLUX : CORRECTED', 'SNOW AMOUNT OVER LAND AFT TSTP KG/M2', 'WET BULB POTENTIAL TEMPERATURE     K', 'V WIND ON PRESSURE LEVELS    B GRID', 'SURFACE SENSIBLE  HEAT FLUX     W/M2', 'BOUNDARY LAYER DEPTH AFTER TIMESTEP', 'CLOUD FRACTION BELOW 1000 FT ASL', 'SURFACE LATENT HEAT FLUX        W/M2', 'CUMULUS-CAPPED BL INDICATOR', 'LOW CLOUD AMOUNT', 'TOTAL COLUMN QCL  RHO GRID', 'FOG FRACTION AT 1.5 M', 'LARGE SCALE RAINFALL RATE    KG/M2/S', 'DOWNWARD LW RAD FLUX: SURFACE', 'RADAR REFLECTIVITY AT 1KM AGL (dBZ)', 'OROGRAPHY (/STRAT LOWER BC)', 'LARGE SCALE RAIN AMOUNT     KG/M2/TS', 'LARGE SCALE SNOWFALL RATE    KG/M2/S', 'TOTAL COLUMN QCF  RHO GRID', '10 METRE WIND V-COMP         B GRID', '

In [46]:
len(unique_long_names)

58

In [47]:
unique_var_names = (list(set(var_names)))
print (unique_var_names)

['STASH_m01s00i024_2', 'STASH_m01s03i217', 'STASH_m01s01i235_2', 'STASH_m01s16i222_2', 'STASH_m01s16i205_2', 'STASH_m01s04i204_3', 'STASH_m01s15i242_2', 'STASH_m01s30i406', 'STASH_m01s30i404', 'STASH_m01s30i461_2', 'STASH_m01s09i203_2', 'STASH_m01s00i031', 'STASH_m01s09i217_2', 'STASH_m01s30i461', 'STASH_m01s03i245_2', 'STASH_m01s00i024', 'STASH_m01s04i204_2', 'STASH_m01s30i404_2', 'STASH_m01s03i250', 'STASH_m01s00i409', 'STASH_m01s02i207_2', 'STASH_m01s03i253', 'STASH_m01s01i208_2', 'STASH_m01s03i226_2', 'STASH_m01s09i204', 'STASH_m01s02i201', 'STASH_m01s03i226', 'STASH_m01s03i310', 'STASH_m01s04i201', 'STASH_m01s00i025', 'STASH_m01s00i033', 'STASH_m01s15i242', 'STASH_m01s09i202_3', 'STASH_m01s08i223', 'STASH_m01s09i202_2', 'STASH_m01s09i205_2', 'STASH_m01s03i281_2', 'STASH_m01s21i104', 'STASH_m01s00i023', 'STASH_m01s15i201', 'STASH_m01s01i207_2', 'STASH_m01s03i225_2', 'STASH_m01s01i202_2', 'STASH_m01s16i204_2', 'STASH_m01s15i142', 'STASH_m01s30i406_2', 'STASH_m01s16i202', 'STASH_m01s

In [48]:
len(unique_var_names)

107

In [59]:
# Compare to STASHC file
STASHC_file='/home/548/pag548/cylc-run/u-dq126/work/20220227T0000Z/Flagship_ERA5to1km_5km_RAL3P2_um_fcst_000/STASHC'

In [60]:
f=open(STASHC_file,'r')

In [15]:
?cubes[0]

Type:           Cube
String form:   
land_binary_mask / (1)              (time: 1; grid_latitude: 976; grid_longitude: 1466)
           Dimen <...>                       'Met Office Unified Model v13.0'
           um_version                  '13.0'
File:           /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/iris/cube.py
Docstring:     
A single Iris cube of data and metadata.

Typically obtained from :func:`iris.load`, :func:`iris.load_cube`,
:func:`iris.load_cubes`, or from the manipulation of existing cubes.

For example:

    >>> cube = iris.load_cube(iris.sample_data_path('air_temp.pp'))
    >>> print(cube)
    air_temperature / (K)               (latitude: 73; longitude: 96)
        Dimension coordinates:
            latitude                             x              -
            longitude                            -              x
        Scalar coordinates:
            forecast_period             6477 hours, bound=(-28083.0, 6477.0) ho

In [16]:
cubes[0].long_name

'LAND MASK (No halo) (LAND=TRUE)'

In [17]:
cubes[0].standard_name

'land_binary_mask'

In [18]:
cubes[0].var_name

'STASH_m01s00i030'

In [24]:
print (cubes[0].attributes['STASH'])

m01s00i030


In [34]:
cubes = iris.load(file)

In [35]:
cubes

[<iris 'Cube' of land_binary_mask / (1) (time: 1; grid_latitude: 976; grid_longitude: 1466)>,
<iris 'Cube' of surface_altitude / (m) (time: 1; grid_latitude: 976; grid_longitude: 1466)>]

In [36]:
file.name

'umnsaa_pa000.nc'